In [17]:
import pandas as pd
import numpy as np
import os
import csv
import email
import re

In [5]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 50C4-BF99

 Directory of c:\Users\gerba\Programming\Flatiron School Data Science Bootcamp - April Cohort 2023\capstone

06/30/2023  01:02 PM    <DIR>          .
06/26/2023  12:47 PM    <DIR>          ..
06/30/2023  01:02 PM             3,265 .gitignore
06/26/2023  12:55 PM                60 capstone.code-workspace
06/30/2023  01:06 PM    <DIR>          data
06/28/2023  02:28 PM           151,822 eda.ipynb
06/30/2023  04:09 PM             3,093 email_preprocessing.ipynb
06/26/2023  12:47 PM            35,823 LICENSE
06/28/2023  03:17 PM             2,100 nb1.ipynb
06/27/2023  10:53 PM               923 README.md
               7 File(s)        197,086 bytes
               3 Dir(s)  1,415,802,413,056 bytes free


In [21]:
def convert_to_eml(root_folder, output_folder):
    for root, dirs, files in os.walk(root_folder):
        for filename in files:
            file_path = os.path.join(root, filename)

            # Process only files (without extension)
            if not os.path.splitext(filename)[1]:
                output_path = os.path.join(output_folder, f'{filename}.eml')

                try:
                    # Read raw text content with UTF-8 encoding
                    with open(file_path, 'r', encoding='utf-8') as file:
                        raw_message = file.read()
                except UnicodeDecodeError:
                    # Skip files not in UTF-8 encoding
                    continue

                # Create an EmailMessage object
                eml = EmailMessage()
                eml.set_content(raw_message)

                # Save as EML file
                with open(output_path, 'wb') as output_file:
                    output_file.write(eml.as_bytes())

# Usage example
root_folder = 'data\enron'
output_folder = 'data\enron_eml'

convert_to_eml(root_folder, output_folder)

In [25]:
def extract_email_info(root_folder, output_csv):
    with open(output_csv, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Index', 'Message ID', 'Message Headers', 'Message Body'])

        index = 1

        for root, dirs, files in os.walk(root_folder):
            for filename in files:
                file_path = os.path.join(root, filename)

                # Process only files with .eml extension
                if os.path.splitext(filename)[1] == '.eml':
                    with open(file_path, 'rb') as file:
                        eml_data = file.read()

                    msg = email.message_from_bytes(eml_data)

                    # Extract Message-ID from the format: Message-ID: <15886558.1075840010371.JavaMail.evans@thyme>
                    message_id_match = re.search(r'<([^<>]+)>', msg.get('Message-ID', ''))
                    message_id = message_id_match.group(1) if message_id_match else ''

                    message_headers = []
                    for key, value in msg.items():
                        header = f'{key}: {value}'
                        message_headers.append(header)

                    message_body = ''
                    if msg.is_multipart():
                        for part in msg.walk():
                            content_type = part.get_content_type()
                            if content_type == 'text/plain':
                                message_body = part.get_payload(decode=True).decode('utf-8')
                                break
                    else:
                        message_body = msg.get_payload(decode=True).decode('utf-8')

                    writer.writerow([index, message_id, '\n'.join(message_headers), message_body])

                    index += 1

# Usage example
root_folder = 'data\enron_eml'
output_csv = 'data\enron_proccessed.csv'

extract_email_info(root_folder, output_csv)

In [2]:
def parse_raw_message(raw_message):
    lines = raw_message.split('\n')
    email = {}
    message = ''
    keys_to_extract = ['from', 'to']
    for line in lines:
        if ':' not in line:
            message += line.strip()
            email['body'] = message
        else:
            pairs = line.split(':')
            key = pairs[0].lower()
            val = pairs[1].strip()
            if key in keys_to_extract:
                email[key] = val
    return email

In [3]:
def parse_into_emails(messages):
    emails = [parse_raw_message(message) for message in messages]
    return {
        'body': map_to_list(emails, 'body'), 
        'to': map_to_list(emails, 'to'), 
        'from_': map_to_list(emails, 'from')
    }